### Задача
Формула для описания изображения с туманом на нём
следующая:

$ I(x) = J(x)t(x) + A(1 - t(x)) $ (1),

где $I(x)$ - наблюдаемая интенсивность в пикселе $x$,
$J(x)$ - излучение сцены,
$A$ - глобальный атмосферный свет,
$t(x)$ - количество света, которое достигает камеры.
Задача убирания тумана с изображения состоит в том,
чтоб вывести $J(x), A, t(x)$ из $I(x)$.
### Метод
Идея метода заключается в следующем наблюдении:
на изображениях без тумана в большинстве патчей
хотя бы один пиксель в одном из цветовых каналов
должен иметь низкую интенсивность.
Формально это записывается так:

$ J ^ {dark}(x) = \min_{c \in \{r, g, b\}}
(\min_{y \in \Omega(x)}(J^{c}(y))) $ (2),

где $J^{c}$ - цветовой канал, $\Omega(x)$ - локальный
патч, с центром в $х$.

Хе в своей работе замечает,
что за исключением неба,
$ J ^ {dark} $ имеет близкие к нулевым значения в
изображениях без тумана. Объясняется это
тем фактом, что на изображениях много теней;
цветных объектов, у которых один из каналов
околонулевой (трава, вода);
тёмных объектов, как стволы деревьев.

Используя эту идею Хе выводит следующее уравнение:

$ \tilde{t}(x) = 1 - \omega \min_{c}(\min_{y \in \Omega(x)}(\frac{I^{c}(y)}{A^{c}}) $
 (3),

Здесь $\omega$ - это параметр сколько тумана надо убрать.
Так как человек оценивает дистанцию в том числе по туману,
если убрать его полностью, изображение может начать
выглядеть неестественно, поэтому небольшая часть
тумана оставляется.

Используя уравнение (3) давайте получим $\tilde{t}(x)$

In [ ]:
import matplotlib.pyplot as plt
from tifffile import imshow
import numpy as np
from skimage import exposure

Я добавил 4 изображения, на которых можно протестировать алгоритм.
В ячейке ниже можно выбрать изображение от 1 до 4, чтоб протестировать на нём алгоритм.

In [ ]:
im_num = 4

image = plt.imread('data/{}.jpg'.format(im_num))
image = image / 255.
imshow(image)

Ниже реализация подсчёта $J^{dark}$. Так как нам надо выбирать
минимальное значение в окне,
то для быстрого подсчёта оконных минимумов я заимплементировал
Marcel van Herk’s fast algorithm для поиска оконных минимумов.

In [ ]:
def dark_channel_prior_faster(img, patch_size=15):
    dark_prior = np.min(img, axis=-1)
    pad = patch_size // 2
    forward = np.zeros(dark_prior.shape[:2])
    backward = np.zeros(dark_prior.shape[:2])
    step = patch_size - 1
    forward[:, :step] = np.minimum.accumulate(dark_prior[:, :step], axis=1)
    backward[:, step - 1:: -1] = np.minimum.accumulate(dark_prior[:, step - 1:: -1], axis=1)
    for j in range(step, img.shape[1], step):
        forward[:, j: j + step] = np.minimum.accumulate(dark_prior[:, j: j + step], axis=1)
        backward[:, j + step - 1: j - 1: -1] = np.minimum.accumulate(dark_prior[:, j + step - 1: j - 1: -1], axis=1)
    forward = np.pad(forward, (0, pad), 'constant', constant_values=1)[:-pad, pad:]
    backward = np.pad(backward, (pad, 0), 'constant', constant_values=1)[pad:, :-pad]
    dark_prior = np.minimum(forward, backward)

    forward = np.zeros(dark_prior.shape[:2])
    backward = np.zeros(dark_prior.shape[:2])
    step = patch_size - 1
    forward[:step] = np.minimum.accumulate(dark_prior[:step], axis=0)
    backward[step - 1:: -1] = np.minimum.accumulate(dark_prior[step - 1:: -1], axis=0)
    for j in range(step, img.shape[1], step):
        forward[j: j + step] = np.minimum.accumulate(dark_prior[j: j + step], axis=0)
        backward[j + step - 1: j - 1: -1] = np.minimum.accumulate(dark_prior[j + step - 1: j - 1: -1], axis=0)
    forward = np.pad(forward, (0, pad), 'constant', constant_values=1)[pad:, :-pad]
    backward = np.pad(backward, (pad, 0), 'constant', constant_values=1)[:-pad, pad:]
    dark_prior = np.minimum(forward, backward)

    return dark_prior

Ниже реализация guided filter, она взята из https://github.com/swehrwein/python-guided-filter
и слегка модифицирована.

In [ ]:
# Реализация guided filter основана на https://github.com/swehrwein/python-guided-filter
def window(img, r):
    (rows, cols) = img.shape[:2]
    im_dst = np.zeros(img.shape)

    tile = [1] * img.ndim
    tile[0] = r
    im_cum = np.cumsum(img, 0)
    im_dst[0:r+1, :, ...] = im_cum[r:2*r+1, :, ...]
    im_dst[r+1:rows-r, :, ...] = im_cum[2*r+1:rows, :, ...] - im_cum[0:rows-2*r-1, :, ...]
    im_dst[rows-r:rows, :, ...] = np.tile(im_cum[rows-1:rows, :, ...], tile) - im_cum[rows-2*r-1:rows-r-1, :, ...]

    tile = [1] * img.ndim
    tile[1] = r
    im_cum = np.cumsum(im_dst, 1)
    im_dst[:, 0:r+1, ...] = im_cum[:, r:2*r+1, ...]
    im_dst[:, r+1:cols-r, ...] = im_cum[:, 2*r+1 : cols, ...] - im_cum[:, 0 : cols-2*r-1, ...]
    im_dst[:, cols-r: cols, ...] = np.tile(im_cum[:, cols-1:cols, ...], tile) - im_cum[:, cols-2*r-1 : cols-r-1, ...]
    return im_dst


def guided_filter(I, p, r, eps):
    N = window(np.ones(I.shape), r)
    mean_I = window(I, r) / N
    mean_p = window(p, r) / N
    var_I = window(I ** 2, r) / N - mean_I ** 2
    a = (window(I * p, r) / N - mean_I * mean_p) / (var_I + eps)
    b = mean_p - a * mean_I

    mean_a = window(a, r) / N
    mean_b = window(b, r) / N

    q = mean_a * I + mean_b
    return q

Для оценки $A$ используются пиксели из наиболее затуманенной
части изображения. Идея в том, что в dark channel
наиболее туманные пиксели будут иметь наибольшие значения,
соответственно чтоб оценить $A$ выбираются топ 0.01%
наиболее интенсивных пикселей в dark channel,
и среди них уже выбираются пиксели с наибольшей интенсивностью
в оригинальном изображении.

Переменная image_normed - это $\frac{I^{c}(y)}{A^{c}}$ в уравнении (3)

In [ ]:
# найдём dark channel
dark_prior = dark_channel_prior_faster(image, 15)

In [ ]:
# выберем топ 0.01% самых интенсивных пикселей в dark channel

mask = dark_prior > (np.quantile(dark_prior, q=0.999) - 0.001)
norm_factor = np.zeros(image.shape[-1])
image_normed = np.zeros(image.shape)

# Найдём наиболее интенсивные пиксели среди выбранных в dark channel
# и отнормируем на них

for i in range(image.shape[-1]):
    norm_factor[i] = np.max(image[..., i][mask])
    image_normed[..., i] = image[..., i] / norm_factor[i]

Теперь, вычислив член $\frac{I^{c}(y)}{A^{c}}$,
найдём $\tilde{t}$ из уравнения (3).

In [ ]:
w = 0.95
t_wave = 1 - w * dark_channel_prior_faster(image_normed, 15)

imshow(t_wave, cmap='Greys')
plt.imsave('data/{}_t_wave.jpg'.format(im_num), t_wave, cmap='Greys')

Как видно на изображении выше, проблема $\tilde{t}$
в возникновении block effects на нём. Чтоб решить эту
проблему применяется guided filter.

После применения фильтра сразу вычисляется
$ \max(t(x), t_{0}) $, где
$t_{0}$ - минимальное количество тумана,
которое мы хотим сохранить.


In [ ]:
# Получим одноканальное изображение, испольльзуем
# его в качестве guide в фильтре
image_normed_guidance = np.mean(image, -1)
# imshow(image_normed_guidance)

In [ ]:
t_0 = 0.1

# Найдём t
t = guided_filter(image_normed_guidance, t_wave, 150, 10**-2)
# Найдём max(t, t_0)
mask = t < t_0
t[mask] = t_0

imshow(t, cmap='Greys')
plt.imsave('data/{}_t.jpg'.format(im_num), t, cmap='Greys')
t = np.expand_dims(t, -1)

Нам известны $A$, $\max(t(x), t_{0})$ и $I(x)$,
остаётся найти $J(x)$. Из формулы (1) легко вывести

$ J(x) = \frac{I(x) - A}{\max(t(x), t_{0})} + A$.

Используя эту формулу получим финальный результат.
Из-за того, что излучение сцены обычно
не столь яркое, как атмосферный свет,
финальное изображение обычно оказывается
затемнённым, поэтому перед показом подправляется
экспозиция.

In [ ]:
refined_image = (image - norm_factor) / t + norm_factor
# imshow(refined_image)

In [ ]:
cutoff = {1: 0.5, 2: 0.4, 3: 0.25, 4: 0.35}

refined_image2 = exposure.rescale_intensity(refined_image, out_range=(0,1))
refined_image2 = exposure.adjust_sigmoid(refined_image2, cutoff=cutoff[im_num], gain=10)
imshow(refined_image2)
imshow(image)
plt.imsave('data/{}_refined.jpg'.format(im_num), refined_image2)